# Programming Assignment: Build a seq2seq model for machine translation.

### Name: Nicolas Jorquera

### Task: Translate English to Spanish

**I built a seq2seq model from scratch. First I had to preprocess a pair of languages; conducting tasks like text cleaning, tokenization, and sequence padding. Then, I implemented both LSTM and Bi-LSTM architectures for the encoder part of the model. This assignment was the focus on Bi-directional LSTMs, which are generally more complex but can capture the context from both directions, making them highly valuable for tasks like machine translation.**

## 1. Data preparation

1. Download data (e.g., "deu-eng.zip") from http://www.manythings.org/anki/
2. Unzip the .ZIP file.
3. Put the .TXT file (e.g., "deu.txt") in the directory "./Data/".

### 1.1. Load and clean text


In [2]:
import re
import string
from unicodedata import normalize
import numpy

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text


# split a loaded document into sentences
def to_pairs(doc):
    lines = doc.strip().split('\n')
    pairs = [line.split('\t') for line in  lines]
    return pairs

def clean_data(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalize unicode characters
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            # tokenize on white space
            line = line.split()
            # convert to lowercase
            line = [word.lower() for word in line]
            # remove punctuation from each token
            line = [word.translate(table) for word in line]
            # remove non-printable chars form each token
            line = [re_print.sub('', w) for w in line]
            # remove tokens with numbers in them
            line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return numpy.array(cleaned)

#### Fill the following blanks:

In [3]:
# e.g., filename = 'Data/deu.txt'
filename = 'spa.txt'

# e.g., n_train = 20000
n_train = 30000

In [4]:
# load dataset
doc = load_doc(filename)

# split into Language1-Language2 pairs
pairs = to_pairs(doc)

# clean sentences
clean_pairs = clean_data(pairs)[0:n_train, :]

In [5]:
for i in range(3000, 3010):
    print('[' + clean_pairs[i, 0] + '] => [' + clean_pairs[i, 1] + ']')

[we like tom] => [tom nos gusta]
[we like tom] => [queremos a tom]
[we like him] => [el nos agrada]
[we like you] => [nos gustas]
[we love tom] => [amamos a tom]
[we love tom] => [nos encanta tom]
[we love you] => [te queremos]
[we love you] => [os queremos]
[we love you] => [te amamos]
[we love you] => [os amamos]


In [6]:
input_texts = clean_pairs[:, 0]
target_texts = ['\t' + text + '\n' for text in clean_pairs[:, 1]]

print('Length of input_texts:  ' + str(input_texts.shape))
print('Length of target_texts: ' + str(input_texts.shape))

Length of input_texts:  (30000,)
Length of target_texts: (30000,)


In [7]:
max_encoder_seq_length = max(len(line) for line in input_texts)
max_decoder_seq_length = max(len(line) for line in target_texts)

print('max length of input  sentences: %d' % (max_encoder_seq_length))
print('max length of target sentences: %d' % (max_decoder_seq_length))

max length of input  sentences: 20
max length of target sentences: 68


**Remark:** To this end, you have two lists of sentences: input_texts and target_texts

## 2. Text processing

### 2.1. Convert texts to sequences

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.

In [8]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# encode and pad sequences
def text2sequences(max_len, lines):
    tokenizer = Tokenizer(char_level=True, filters='')
    tokenizer.fit_on_texts(lines)
    seqs = tokenizer.texts_to_sequences(lines)
    seqs_pad = pad_sequences(seqs, maxlen=max_len, padding='post')
    return seqs_pad, tokenizer.word_index


encoder_input_seq, input_token_index = text2sequences(max_encoder_seq_length, 
                                                      input_texts)
decoder_input_seq, target_token_index = text2sequences(max_decoder_seq_length, 
                                                       target_texts)

print('shape of encoder_input_seq: ' + str(encoder_input_seq.shape))
print('shape of input_token_index: ' + str(len(input_token_index)))
print('shape of decoder_input_seq: ' + str(decoder_input_seq.shape))
print('shape of target_token_index: ' + str(len(target_token_index)))

shape of encoder_input_seq: (30000, 20)
shape of input_token_index: 27
shape of decoder_input_seq: (30000, 68)
shape of target_token_index: 29


In [9]:
num_encoder_tokens = len(input_token_index) + 1
num_decoder_tokens = len(target_token_index) + 1

print('num_encoder_tokens: ' + str(num_encoder_tokens))
print('num_decoder_tokens: ' + str(num_decoder_tokens))

num_encoder_tokens: 28
num_decoder_tokens: 30


**Remark:** To this end, the input language and target language texts are converted to 2 matrices. 

- Their number of rows are both n_train.
- Their number of columns are respective max_encoder_seq_length and max_decoder_seq_length.

The followings print a sentence and its representation as a sequence.

In [10]:
target_texts[100]

'\tse\n'

In [11]:
decoder_input_seq[100, :]

array([6, 5, 2, 7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

## 2.2. One-hot encode

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.
- It is represented by a $n\times t \times v$ tensor ($t$ is the number of unique chars) after the one-hot encoding.

In [13]:
from keras.utils import to_categorical

# one hot encode target sequence
def onehot_encode(sequences, max_len, vocab_size):
    n = len(sequences)
    data = numpy.zeros((n, max_len, vocab_size))
    for i in range(n):
        data[i, :, :] = to_categorical(sequences[i], num_classes=vocab_size)
    return data

encoder_input_data = onehot_encode(encoder_input_seq, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data = onehot_encode(decoder_input_seq, max_decoder_seq_length, num_decoder_tokens)

decoder_target_seq = numpy.zeros(decoder_input_seq.shape)
decoder_target_seq[:, 0:-1] = decoder_input_seq[:, 1:]
decoder_target_data = onehot_encode(decoder_target_seq, 
                                    max_decoder_seq_length, 
                                    num_decoder_tokens)

print(encoder_input_data.shape)
print(decoder_input_data.shape)

(30000, 20, 28)
(30000, 68, 30)


## 3. Build the networks (for training)

- Build encoder, decoder, and connect the two modules to get "model". 

- Fit the model on the bilingual data to train the parameters in the encoder and decoder.

### 3.1. Encoder network

- Input:  one-hot encode of the input language

- Return: 

    -- output (all the hidden states   $h_1, \cdots , h_t$) are always discarded
    
    -- the final hidden state  $h_t$
    
    -- the final conveyor belt $c_t$

In [14]:
from keras.layers import Input, LSTM, Bidirectional, Concatenate
from keras.models import Model

latent_dim = 256

# inputs of the encoder network
encoder_inputs = Input(shape=(None, num_encoder_tokens), 
                       name='encoder_inputs')

# set the Bidirectional LSTM layer
encoder_bilstm = Bidirectional(LSTM(latent_dim, return_state=True, 
                                    dropout=0.5, name='encoder_lstm'))
_, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_inputs)

# Concatenate the forward and backward states
state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

# build the encoder network model with the concatenated states
encoder_model = Model(inputs=encoder_inputs, 
                      outputs=[state_h, state_c],
                      name='encoder')


Print a summary and save the encoder network structure to "./encoder.pdf"

In [15]:
from IPython.display import SVG
from tensorflow.keras.utils import model_to_dot, plot_model

SVG(model_to_dot(encoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=encoder_model, show_shapes=False,
)

encoder_model.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_inputs (InputLayer  [(None, None, 28)]           0         []                            
 )                                                                                                
                                                                                                  
 bidirectional (Bidirection  [(None, 512),                583680    ['encoder_inputs[0][0]']      
 al)                          (None, 256),                                                        
                              (None, 256),                                                        
                              (None, 256),                                                        
                              (None, 256)]                                                  

### 3.2. Decoder network

- Inputs:  

    -- one-hot encode of the target language
    
    -- The initial hidden state $h_t$ 
    
    -- The initial conveyor belt $c_t$ 

- Return: 

    -- output (all the hidden states) $h_1, \cdots , h_t$

    -- the final hidden state  $h_t$ (discarded in the training and used in the prediction)
    
    -- the final conveyor belt $c_t$ (discarded in the training and used in the prediction)

In [35]:
from keras.layers import Input, LSTM, Dense
from keras.models import Model

# Double the latent dimensions to match the concatenated states from the Bidirectional LSTM
decoder_latent_dim = 2 * latent_dim

# inputs of the decoder network
decoder_input_h = Input(shape=(decoder_latent_dim,), name='decoder_input_h')
decoder_input_c = Input(shape=(decoder_latent_dim,), name='decoder_input_c')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# set the LSTM layer with doubled latent dimensions
decoder_lstm = LSTM(decoder_latent_dim, return_sequences=True, 
                    return_state=True, dropout=0.5, name='decoder_lstm')
decoder_lstm_outputs, state_h, state_c = decoder_lstm(decoder_input_x, 
                                                      initial_state=[decoder_input_h, decoder_input_c])

# set the dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

# build the decoder network model
decoder_model = Model(inputs=[decoder_input_x, decoder_input_h, decoder_input_c],
                      outputs=[decoder_outputs, state_h, state_c],
                      name='decoder')


Print a summary and save the encoder network structure to "./decoder.pdf"

In [36]:
from IPython.display import SVG
from tensorflow.keras.utils import model_to_dot, plot_model

SVG(model_to_dot(decoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=decoder_model, show_shapes=False,
    to_file='decoder.pdf'
)

decoder_model.summary()

Model: "decoder"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 decoder_input_x (InputLaye  [(None, None, 30)]           0         []                            
 r)                                                                                               
                                                                                                  
 decoder_input_h (InputLaye  [(None, 512)]                0         []                            
 r)                                                                                               
                                                                                                  
 decoder_input_c (InputLaye  [(None, 512)]                0         []                            
 r)                                                                                         

### 3.3. Connect the encoder and decoder

In [37]:
# input layers
encoder_input_x = Input(shape=(None, num_encoder_tokens), name='encoder_input_x')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# connect encoder to decoder
encoder_final_states = encoder_model([encoder_input_x])

# Ensure that the decoder LSTM's hidden units match the dimensions of the concatenated states from the Bi-LSTM
decoder_lstm = LSTM(2 * latent_dim, return_sequences=True, return_state=True, dropout=0.5, name='decoder_lstm')
decoder_lstm_output, _, _ = decoder_lstm(decoder_input_x, initial_state=encoder_final_states)
decoder_pred = decoder_dense(decoder_lstm_output)

model = Model(inputs=[encoder_input_x, decoder_input_x], 
              outputs=decoder_pred, 
              name='model_training')

In [38]:
print(state_h)
print(decoder_input_h)

KerasTensor(type_spec=TensorSpec(shape=(None, 512), dtype=tf.float32, name=None), name='decoder_lstm/PartitionedCall:2', description="created by layer 'decoder_lstm'")
KerasTensor(type_spec=TensorSpec(shape=(None, 512), dtype=tf.float32, name='decoder_input_h'), name='decoder_input_h', description="created by layer 'decoder_input_h'")


In [39]:
from IPython.display import SVG

SVG(model_to_dot(model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=model, show_shapes=False,
    to_file='model_training.pdf'
)

model.summary()

Model: "model_training"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 encoder_input_x (InputLaye  [(None, None, 28)]           0         []                            
 r)                                                                                               
                                                                                                  
 decoder_input_x (InputLaye  [(None, None, 30)]           0         []                            
 r)                                                                                               
                                                                                                  
 encoder (Functional)        [(None, 512),                583680    ['encoder_input_x[0][0]']     
                              (None, 512)]                                           

### Issues Encountered:
- Their was a mismatch in the dimensionality between the encoder and the decoder when I replaced the unidirectional LSTM in the encoder with a Bidirectional LSTM. This change doubled the dimensions of the hidden and cell states; so I had to modify the decoder accordingly. 
- Not sure if normal, but model is going to take over 2 hours to train; utilizing a fairly efficient gaming laptop.

### 3.5. Fit the model on the bilingual dataset

- encoder_input_data: one-hot encode of the input language

- decoder_input_data: one-hot encode of the input language

- decoder_target_data: labels (left shift of decoder_input_data)

- tune the hyper-parameters

- stop when the validation loss stop decreasing.

In [40]:
print('shape of encoder_input_data' + str(encoder_input_data.shape))
print('shape of decoder_input_data' + str(decoder_input_data.shape))
print('shape of decoder_target_data' + str(decoder_target_data.shape))

shape of encoder_input_data(30000, 20, 28)
shape of decoder_input_data(30000, 68, 30)
shape of decoder_target_data(30000, 68, 30)


In [41]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

model.fit([encoder_input_data, decoder_input_data],  # training data
          decoder_target_data,                       # labels (left shift of the target sequences)
          batch_size=64, epochs=50, validation_split=0.2)

model.save('seq2seq.h5')

Epoch 1/50
375/375 [==============================] - 392s 1s/step - loss: 0.8206 - val_loss: 0.8369
Epoch 2/50
375/375 [==============================] - 376s 1s/step - loss: 0.6722 - val_loss: 0.7255
Epoch 3/50
375/375 [==============================] - 242s 644ms/step - loss: 0.6391 - val_loss: 0.6993
Epoch 4/50
375/375 [==============================] - 155s 412ms/step - loss: 0.6234 - val_loss: 0.6764
Epoch 5/50
375/375 [==============================] - 154s 412ms/step - loss: 0.6129 - val_loss: 0.6594
Epoch 6/50
375/375 [==============================] - 156s 417ms/step - loss: 0.6034 - val_loss: 0.6436
Epoch 7/50
375/375 [==============================] - 154s 410ms/step - loss: 0.5943 - val_loss: 0.6324
Epoch 8/50
375/375 [==============================] - 166s 442ms/step - loss: 0.5866 - val_loss: 0.6169
Epoch 9/50
375/375 [==============================] - 158s 423ms/step - loss: 0.5789 - val_loss: 0.6051
Epoch 10/50
375/375 [==============================] - 155s 414ms/step

C:\Users\nicol\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## 4. Make predictions


### 4.1. Translate English to XXX

1. Encoder read a sentence (source language) and output its final states, $h_t$ and $c_t$.
2. Take the [star] sign "\t" and the final state $h_t$ and $c_t$ as input and run the decoder.
3. Get the new states and predicted probability distribution.
4. sample a char from the predicted probability distribution
5. take the sampled char and the new states as input and repeat the process (stop if reach the [stop] sign "\n").

In [42]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [43]:
def decode_sequence(input_seq):
    states_value = encoder_model.predict(input_seq)

    target_seq = numpy.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # this line of code is greedy selection
        # try to use multinomial sampling instead (with temperature)
        sampled_token_index = numpy.argmax(output_tokens[0, -1, :])
        
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = numpy.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence


In [44]:
for seq_index in range(2100, 2120):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('English:       ', input_texts[seq_index])
    print('Spanish (true): ', target_texts[seq_index][1:-1])
    print('Spanish (pred): ', decoded_sentence[0:-1])


1/1 [==============================] - 0s 17ms/step
-
English:        dig quickly
German (true):  cave rapido
German (pred):  yyyyyb   a aaaaaattststststststststststststststststststststststststs
1/1 [==============================] - 0s 16ms/step
-
English:        dig quickly
German (true):  caven rapido
German (pred):  yyyyyb   a aaaaaattststststststststststststststststststststststststs
1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 17ms/step
-
English:        do as i say
German (true):  haz lo que te digo
German (pred):  tttttttnnaaaaaaaanaaattttststtststststststststststststststststststst
1/1 [==============================] - 0s 15ms/step
-
English:        do it again
German (true):  hacedlo de nuevo
German (pred):  ttttttttaaaaaaaaaaattttststtstststststststststststststststststststst
1/1 [==============================] - 0s 16ms/step
-
English:        do it again
German (true):  hazlo otra vez
German (pred):  ttttttttaaaaaaaaaaattttststtstststststststststststststststststststst
1/1 [==============================] - 0s 15ms/step
-
English:        do it later
German (true):  hazlo ahorita
German (pred):  tttttttsssssaaatstststststststststststststststststststststststststst
1/1 [==============================] - 0s 17ms/step


1/1 [==============================] - 0s 16ms/step
-
English:        do it later
German (true):  hazlo al ratito
German (pred):  tttttttsssssaaatstststststststststststststststststststststststststst
1/1 [==============================] - 0s 17ms/step
-
English:        do you bowl
German (true):  juegas boliche
German (pred):  tttnnnaaaaaaaaaaaanaaanaattttstststtstststststststststststststststst
1/1 [==============================] - 0s 15ms/step


1/1 [==============================] - 0s 16ms/step
-
English:        dont argue
German (true):  no discutais
German (pred):  nnnnnnnaaaaaaaaaaanaaanaaattttstststtstststststststststststststststs
1/1 [==============================] - 0s 17ms/step
-
English:        dont blink
German (true):  no parpadees
German (pred):  nnnnnnaaaaaaaaaaaaanaaatttsttstststststststststststststststststststs
1/1 [==============================] - 0s 16ms/step
-
English:        dont cheat
German (true):  no copieis
German (pred):  nnnnnnaaaaaaaaaaaanaaatttsttstststststststststststststststststststst
1/1 [==============================] - 0s 16ms/step
-
English:        dont do it
German (true):  no lo hagas
German (pred):  nnnnnnaaaaaaaaaaaanaaanaaattttststtststststststststststststststststs
1/1 [==============================] - 0s 15ms/step
-
English:        dont do it
German (true):  no lo haga
German (pred):  nnnnnnaaaaaaaaaaaanaaanaaattttststtststststststststststststststststs


1/1 [==============================] - 0s 17ms/step
-
English:        dont do it
German (true):  no lo hagas
German (pred):  nnnnnnaaaaaaaaaaaanaaanaaattttststtststststststststststststststststs
1/1 [==============================] - 0s 15ms/step
-
English:        dont do it
German (true):  no lo hagais
German (pred):  nnnnnnaaaaaaaaaaaanaaanaaattttststtststststststststststststststststs
1/1 [==============================] - 0s 15ms/step
-
English:        dont do it
German (true):  no lo hagan
German (pred):  nnnnnnaaaaaaaaaaaanaaanaaattttststtststststststststststststststststs
1/1 [==============================] - 0s 16ms/step
-
English:        dont do it
German (true):  no lo haga
German (pred):  nnnnnnaaaaaaaaaaaanaaanaaattttststtststststststststststststststststs
1/1 [==============================] - 0s 16ms/step


1/1 [==============================] - 0s 17ms/step
-
English:        dont fight
German (true):  no os peleeis
German (pred):  nnnnnnaaaaaaaaaaaanaaatttsttstststststststststststststststststststst
1/1 [==============================] - 0s 17ms/step
-
English:        dont fight
German (true):  no peleen
German (pred):  nnnnnnaaaaaaaaaaaanaaatttsttstststststststststststststststststststst
1/1 [==============================] - 0s 15ms/step
-
English:        dont fight
German (true):  no pelees
German (pred):  nnnnnnaaaaaaaaaaaanaaatttsttstststststststststststststststststststst


### 4.2. Translate an English sentence to the target language

1. Tokenization
2. One-hot encode
3. Translate

In [48]:
# example:
input_sentence = 'I hope this work'

input_sequence, _ = text2sequences(max_encoder_seq_length, [input_sentence])

input_x = onehot_encode(input_sequence, max_encoder_seq_length, num_encoder_tokens)

translated_sentence = decode_sequence(input_x)

print('source sentence is: ' + input_sentence)
print('translated sentence is: ' + translated_sentence)

1/1 [==============================] - 0s 18ms/step
source sentence is: I hope this work
translated sentence is: tttttttttttttst tttsttsttststtstststststststststststststststststststs
